## 분석목표
- '지저분한' 데이터를 처리하는 경험을 쌓고, 실제 현장에서 여러가지 상황에 대처할 수 있는 능력 키우기

## 데이터 읽기

In [1]:
import pandas as pd

#매출이력 데이터
uriage_data=pd.read_csv('uriage.csv')
uriage_data.head()

C:\Users\taeeu\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\taeeu\anaconda3\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
C:\Users\taeeu\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [2]:
kokyaku_data=pd.read_excel('kokyaku_daicho.xlsx')
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


## 데이터 오류 살펴보기
### -> 상품A, 상 품 a, 상품a등 다른 상품으로 집계

In [3]:
uriage_data['item_name'].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

### -> 결측치 NaN을 확인

In [4]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

## 상품명 오류 수정하기

In [5]:
print(len(pd.unique(uriage_data['item_name'])))

99


In [6]:
#전부 대문자로 변환
uriage_data['item_name']=uriage_data['item_name'].str.upper()

In [7]:
#공백 삭제
uriage_data['item_name']=uriage_data['item_name'].str.replace(' ', '')
uriage_data['item_name']=uriage_data['item_name'].str.replace('  ', '')

In [8]:
#오름차순으로 정렬
uriage_data.sort_values(by=['item_name'], ascending=True)

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1748,2019-05-19 20:22,상품A,100.0,김시훈
223,2019-06-25 8:13,상품A,100.0,김유진
1742,2019-06-13 16:03,상품A,100.0,김건희
1738,2019-02-10 0:28,상품A,100.0,김하랑
...,...,...,...,...
2880,2019-04-22 0:36,상품Y,NaN,김동욱
2881,2019-04-30 14:21,상품Y,NaN,김하준
1525,2019-01-24 10:27,상품Y,2500.0,김범준
1361,2019-05-28 13:45,상품Y,2500.0,김수현


In [9]:
print(len(pd.unique(uriage_data['item_name'])))
#99->26개로 오류를 수정한 것을 확인

26


## 금액의 결측치 수정

In [11]:
uriage_data.isnull().any(axis=0)

purchase_date    False
item_name        False
item_price        True
customer_name    False
dtype: bool

In [16]:
#결측치를 저장, 몇행에 결측치가 있는지 저장
fig_is_null=uriage_data['item_price'].isnull()
fig_is_null=fig_is_null.copy()

#결측치인 상품 추출
#unique() : 추출한 상품명에서 중복을 제거
#list() : 변수의 값을 리스트 형식으로 변환
#loc : 조건에 일치하는 데이터를 추출
for trg in list(uriage_data.loc[fig_is_null, 'item_name'].unique()):
    price=uriage_data.loc[(~fig_is_null)&(uriage_data['item_name']==trg), 'item_price'].max()
    uriage_data['item_price'].loc[(fig_is_null)&(uriage_data['item_name']==trg)]=price
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상품S,1900.0,김우찬
2,2019-05-11 19:42,상품A,100.0,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품A,100.0,김강현


In [17]:
#NaN이 사라졌나 확인
uriage_data.isnull().any(axis=0)

purchase_date    False
item_name        False
item_price       False
customer_name    False
dtype: bool

In [19]:
#금액이 정상적으로 수정되었는지 확인
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg + '의 최고가 : '+str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].max())+'의 최저가 : '+str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].min(skipna=False)))

상품A의 최고가 : 100.0의 최저가 : 100.0
상품B의 최고가 : 200.0의 최저가 : 200.0
상품C의 최고가 : 300.0의 최저가 : 300.0
상품D의 최고가 : 400.0의 최저가 : 400.0
상품E의 최고가 : 500.0의 최저가 : 500.0
상품F의 최고가 : 600.0의 최저가 : 600.0
상품G의 최고가 : 700.0의 최저가 : 700.0
상품H의 최고가 : 800.0의 최저가 : 800.0
상품I의 최고가 : 900.0의 최저가 : 900.0
상품J의 최고가 : 1000.0의 최저가 : 1000.0
상품K의 최고가 : 1100.0의 최저가 : 1100.0
상품L의 최고가 : 1200.0의 최저가 : 1200.0
상품M의 최고가 : 1300.0의 최저가 : 1300.0
상품N의 최고가 : 1400.0의 최저가 : 1400.0
상품O의 최고가 : 1500.0의 최저가 : 1500.0
상품P의 최고가 : 1600.0의 최저가 : 1600.0
상품Q의 최고가 : 1700.0의 최저가 : 1700.0
상품R의 최고가 : 1800.0의 최저가 : 1800.0
상품S의 최고가 : 1900.0의 최저가 : 1900.0
상품T의 최고가 : 2000.0의 최저가 : 2000.0
상품U의 최고가 : 2100.0의 최저가 : 2100.0
상품V의 최고가 : 2200.0의 최저가 : 2200.0
상품W의 최고가 : 2300.0의 최저가 : 2300.0
상품X의 최고가 : 2400.0의 최저가 : 2400.0
상품Y의 최고가 : 2500.0의 최저가 : 2500.0
상품Z의 최고가 : 2600.0의 최저가 : 2600.0


## 고객 이름 오류 수정하기

In [21]:
kokyaku_data['고객이름'].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [22]:
uriage_data['customer_name'].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

In [24]:
kokyaku_data['고객이름']=kokyaku_data['고객이름'].str.replace(' ', '')
kokyaku_data['고객이름']=kokyaku_data['고객이름'].str.replace('  ', '')
kokyaku_data['고객이름'].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

## 날짜 오류 수정하기

In [28]:
fig_is_serial=kokyaku_data['등록일'].astype('str').str.isdigit()
fig_is_serial.sum()

22